## Model 2


Stochastic optimization model used to solve the two-stage problem

In [97]:
using JuMP,HiGHS,DataFrames
using CSV
using DataFrames

In [98]:
# Import data about buses
df_buses = CSV.read("data/24_buses.csv", DataFrame)

Row,Node,I,P_max,C,C_RU,C_RD,C_U,C_D,J,V_LOL,Q,C_cur,Lambda,b,LC_Max
,Int64,String3,String7,String7,String7,String7,String7,String7,Int64,Int64,Int64,Int64?,Int64,String15,String7
1,1,1;2,50;110,10;30,16;13,15;12,10;30,10;30,1,200,1,0,2,1;7.69230769231,0;100
2,2,1,100,35,10,9,35,35,1,200,0,missing,1,7.69230769231;1,100;0


In [99]:
"""
Transforms indices into range for lists of objects (such as generation units)

Input of 0 or empty cells is translated to no objects (i.e. empty array)
"""
function string_to_range(s)
    # Check if s is missing or a string that represents an empty cell
    if s == "missing" || s == "0"
        return Int[]  # Return an empty array of Int type
    else
        str = string(s)  # Convert to string if not already
        nums = split(str, ";")  # Split the string by comma
        nums = parse.(Int, nums)  # Parse each part to an integer
        return minimum(nums):maximum(nums)  # Return the range
    end
end

"""
Transforms indices into array of values (such as generation costs per unit for each generation unit)

Input of missing cells is translated to no values (i.e. empty array)

Input of 0 is translated to 0 since it can be a coefficient
"""
function string_to_array(s)
    # Check if s is missing or a string that represents an empty cell
    #if s == "0"
    #    return 0  # Return an empty array of Float64 type

    if s == "missing"
        return Float64[]

    else
        str = string(s)  # Convert to string if not already
        nums = split(str, ";")  # Split the string by semicolon
        nums = parse.(Float64, nums)  # Parse each part to a float
        return nums  # Return the array of floats
    end
end


string_to_array

In [100]:
# Transform columns with objects
columns_to_range = [:I, :J, :Q]

for col in columns_to_range
    # Convert the column to String if it's not already
    df_buses[!, col] = string.(df_buses[!, col])
    # Apply the function to the column
    df_buses[!, col] = [string_to_range(value) for value in df_buses[!, col]]
end

# Transform columns with values
columns_to_array = [:C, :P_max, :C_RU, :C_RD, :C_U,	:C_D, :V_LOL, :C_cur, :b, :LC_Max, :Lambda]
for col in columns_to_array
    # Convert the column to String if it's not already
    df_buses[!, col] = string.(df_buses[!, col])
    # Apply the function to the column
    df_buses[!, col] = [string_to_array(value) for value in df_buses[!, col]]
end


In [101]:
df_buses

Row,Node,I,P_max,C,C_RU,C_RD,C_U,C_D,J,V_LOL,Q,C_cur,Lambda,b,LC_Max
,Int64,UnitRang…,Array…,Array…,Array…,Array…,Array…,Array…,UnitRang…,Array…,Abstract…,Array…,Array…,Array…,Array…
1,1,1:2,"[50.0, 110.0]","[10.0, 30.0]","[16.0, 13.0]","[15.0, 12.0]","[10.0, 30.0]","[10.0, 30.0]",1:1,[200.0],1:1,[0.0],[2.0],"[1.0, 7.69231]","[0.0, 100.0]"
2,2,1:1,[100.0],[35.0],[10.0],[9.0],[35.0],[35.0],1:1,[200.0],Int64[],Float64[],[1.0],"[7.69231, 1.0]","[100.0, 0.0]"


In [102]:
# Import the rest of the data
df_other = CSV.read("data/other_data.csv", DataFrame)

Row,T,Omega,Pi
,String3,String3,String7
1,1;1,1;2,0.6;0.4


In [103]:
# Transform columns with objects
columns_to_range = [:T, :Omega]

for col in columns_to_range
    # Convert the column to String if it's not already
    df_other[!, col] = string.(df_other[!, col])
    # Apply the function to the column
    df_other[!, col] = [string_to_range(value) for value in df_other[!, col]]
end

# Transform columns with values
columns_to_array = [:Pi]
for col in columns_to_array
    # Convert the column to String if it's not already
    df_other[!, col] = string.(df_other[!, col])
    # Apply the function to the column
    df_other[!, col] = [string_to_array(value) for value in df_other[!, col]]
end

In [104]:
df_other

Row,T,Omega,Pi
,UnitRang…,UnitRang…,Array…
1,1:1,1:2,"[0.6, 0.4]"


In [105]:
"""
N = 1:2                                #number of nodes (buses)
I=[[1:2]; [1:1]]                           #number of generation units of node n
# Q [0:0] 
Q = [[1], []]                          # Q[n] gives the wind units at node n
T=1:1                                  #number of time intervalls 
Ω=1:2                                  #number of scenarios
J=[[1:1]; [1:1]]                           #number of loads  of node n
Λ=[[2:2]; [1:1]]                            #set of nodes directly connected to node n
#Λ=[[2] [1]]                            #set of nodes directly connected to node n
C=[[10, 30], [35]]                          #unit cost of generation C[N,I]
C_RU=[[16, 13], [10]]                       #cost of up reserve commitment C_RU[N,I]
C_RD=[[15, 12], [9]]                        #cost of down reserve commitment C_RD[N,I]
C_U=[[10, 30], [35]]                        #cost of up reserve commitment C_U[N,I]
C_D=[[10, 30], [35]]                        #cost of down reserve commitment C_D[N,I]
π=[0.6, 0.4]                           #probabilities of the scenarios
V_LOL=[[200], [200]]                        #value of the lost load
C_cur=[[0]] #[15]                         #cost of curtailment at turbine C_cur[N,Q]
L= [[40], [100]]               #loads for every time intervall L=[N] [J,T]
#W_realized = [[[50] [10]] [[50] [10]] [[50] [10]]] #create random scenarios for four time points q,ω,t

# changed back to original
W_realized = [[[[50],[10]],[]],[]] #70 .+ 15 * randn(1, 1, 2, 1)  #create random scenarios for n,q,ω,t [[[[50],[10]]],[]]#
#[[[111 112] [121 122]] [[211 212] [221 222]] [[311 312] [321 322]]]
P_max=[[50, 110], [100]];                   #maximum production capacity

# From IEEE 24 bus
b = [[1, 1/0.13], [1/0.13, 1]]                     #line susceptance (loss of transmission)
LC_Max = [[0, 100], [100, 0]]          #maximum transmission quantity between two buses [n][ℓ]
print(W_realized[1][1][2][1])
"""

"N = 1:2                                #number of nodes (buses)\nI=[[1:2]; [1:1]]                           #number of generation units of node n\n# Q [0:0] \nQ = [[1], []]                          # Q[n] gives the wind units at node n\nT=1:1                                " ⋯ 1418 bytes ⋯ "    #maximum production capacity\n\n# From IEEE 24 bus\nb = [[1, 1/0.13], [1/0.13, 1]]                     #line susceptance (loss of transmission)\nLC_Max = [[0, 100], [100, 0]]          #maximum transmission quantity between two buses [n][ℓ]\nprint(W_realized[1][1][2][1])\n"

In [106]:
"""
data= Dict(
    :N => N,
    :I => I,
    :Q => Q,
    :T => T,
    :Ω => Ω,
    :J => J,
    :Λ => Λ,
    :C => C,
    :C_RU => C_RU,
    :C_RD => C_RD,
    :C_U => C_U,
    :C_D => C_D,
    :π => π,
    :V_LOL => V_LOL,
    :C_cur => C_cur,
    :L => L,
    :W_realized => W_realized,
    :P_max => P_max,
    :b => b,
    :LC_Max => LC_Max
    );
#print(W_realized[1,2,1])
"""

"data= Dict(\n    :N => N,\n    :I => I,\n    :Q => Q,\n    :T => T,\n    :Ω => Ω,\n    :J => J,\n    :Λ => Λ,\n    :C => C,\n    :C_RU => C_RU,\n    :C_RD => C_RD,\n    :C_U => C_U,\n    :C_D => C_D,\n    :π => π,\n    :V_LOL => V_LOL,\n    :C_cur => C_cur,\n    :L => L,\n    :W_realized => W_realized,\n    :P_max => P_max,\n    :b => b,\n    :LC_Max => LC_Max\n    );\n#print(W_realized[1,2,1])\n"

In [107]:
# Create an empty dictionary to hold all nodes
data = Dict()

# Iterate over the range of nodes
for node in 1:2
    # Create a dictionary for the current node with all other variables
    data_node = Dict(
        :I => df_buses.I[node],
        :Q => df_buses.Q[node],
#        :T => df_other.T,
#        :Ω => df_other.Omega,
        :J => df_buses.J[node],
        :Λ => df_buses.Lambda[node],
        :C => df_buses.C[node],
        :C_RU => df_buses.C_RU[node],
        :C_RD => df_buses.C_RD[node],
        :C_U => df_buses.C_U[node],
        :C_D => df_buses.C_D[node],
#        :π => df_other.Pi,
        :V_LOL => df_buses.V_LOL[node],
        :C_cur => df_buses.C_cur[node],
#        :L => df_buses.L[node],
#        :W_realized => W_realized,
        :P_max => df_buses.P_max[node],
        :b => df_buses.b[node],
        :LC_Max => df_buses.LC_Max[node],
        )
    
    # Assign the dictionary to the current node key
    data[node] = data_node
end



In [108]:
println(data[1])
println(data[2])

Dict{Symbol, AbstractVector}(:C_RU => [16.0, 13.0], :b => [1.0, 7.69230769231], :C_U => [10.0, 30.0], :C_D => [10.0, 30.0], :P_max => [50.0, 110.0], :C_cur => [0.0], :LC_Max => [0.0, 100.0], :Λ => [2.0], :C_RD => [15.0, 12.0], :Q => 1:1, :V_LOL => [200.0], :I => 1:2, :J => 1:1, :C => [10.0, 30.0])
Dict{Symbol, AbstractVector}(:C_RU => [10.0], :b => [7.69230769231, 1.0], :C_U => [35.0], :C_D => [35.0], :P_max => [100.0], :C_cur => Float64[], :LC_Max => [100.0, 0.0], :Λ => [1.0], :C_RD => [9.0], :Q => Int64[], :V_LOL => [200.0], :I => 1:1, :J => 1:1, :C => [35.0])


In [110]:
function init_variables(model::JuMP.Model)
    P=@variable(model, P[n in data[:N], i in data[:I][n], t in data[:T]] >=0) #energy generated
    R_U=@variable(model, R_U[n in data[:N], i in data[:I][n], t in data[:T]] >=0) #committed upward reserve capacity of generator i
    R_D=@variable(model, R_D[n in data[:N], i in data[:I][n], t in data[:T]] >=0) #committed downward reserve capacity of generator i
    r_U=@variable(model, r_U[n in data[:N], i in data[:I][n], ω in data[:Ω], t in data[:T]] >=0) #up regulation of generator i in case 𝜔
    r_D=@variable(model, r_D[n in data[:N], i in data[:I][n], ω in data[:Ω], t in data[:T]] >=0) #down regulation of generator i in case 𝜔
    L_Shed=@variable(model, L_Shed[n in data[:N], j in data[:J][n], ω in data[:Ω], t in data[:T]] >=0) #loss of load at load demand j in case 𝜔
    W_spill=@variable(model, W_spill[n in data[:N], q in data[:Q][n], ω in data[:Ω], t in data[:T]] >=0) #curtailment of turbine q in case 𝜔
    W_s=@variable(model, W_s[n in data[:N], q in data[:Q][n], t in data[:T]] >=0) #scheduled wind power generation at turbine q
    𝛿=@variable(model, 𝛿[n in data[:N], ℓ in data[:Λ][n], t in data[:T]] >= 0) #voltage angle
    PF=@variable(model, PF[n in data[:N], ℓ in data[:Λ][n], t in data[:T]] >= 0) #power flow          # added >= 0 
    PF_scenario=@variable(model, PF_scenario[ω in data[:Ω], n in data[:N], ℓ in data[:Λ][n], t in data[:T]] >= 0) #power flow scenario
    
    vars= Dict(
        :P => P,
        :R_U => R_U,
        :R_D => R_D,
        :r_U => r_U,
        :r_D => r_D,
        :L_Shed => L_Shed,
        :W_spill => W_spill,
        :W_s => W_s,            # added commas
        :𝛿 => 𝛿,
        :PF => PF,
        :PF_scenario => PF_scenario
    )
    
    return vars
end;


In [111]:
function init_constraints(model::JuMP.Model, data::Dict, vars::Dict)
    @constraints(model, begin 
            c1[n in data[:N],i in data[:I][n], t in data[:T]], vars[:P][n, i, t]+ vars[:R_U][n, i, t]<= data[:P_max][n][i]
            c2[n in data[:N],i in data[:I][n] ,t in data[:T]], vars[:P][n, i, t]- vars[:R_D][n, i, t]>= 0
            c3[n in data[:N],i in data[:I][n], t in data[:T], ω in data[:Ω]], vars[:r_U][n, i, ω, t] <= vars[:R_U][n,i,t]
            c4[n in data[:N],i in data[:I][n], t in data[:T], ω in data[:Ω]], vars[:r_D][n, i, ω, t] <= vars[:R_D][n,i,t]
            
            c5[ω in data[:Ω], n in data[:N], ℓ in data[:Λ][n], t in data[:T]], vars[:PF_scenario][ω, n, ℓ, t] <= data[:LC_Max][n][ℓ]
            
            c6[n in data[:N], ℓ in data[:Λ][n], t in data[:T]], vars[:PF][n, ℓ, t] <= data[:LC_Max][n][ℓ]
            # Something wrong with powerflow
            c7[ω in data[:Ω], n in data[:N], ℓ in data[:Λ][n], t in data[:T]], (vars[:𝛿][n, ℓ, t] - vars[:𝛿][ℓ, n, t]) * data[:b][n][ℓ] == vars[:PF][n, ℓ, t]

            ## !!
            Power_balance_day_ahead[n in data[:N], t in data[:T]], 
            sum(vars[:P][n,i,t] for i in data[:I][n]) + sum(vars[:W_s][n,q,t] for q in data[:Q][n]) - sum(data[:L][n][j,t] for j in data[:J][n]) - sum(vars[:PF][n, ℓ, t] for ℓ in data[:Λ][n]) == 0

            Power_balance_at_stage[ω in data[:Ω], n in data[:N], t in data[:T]], 
            sum(vars[:r_U][n,i, ω, t] for i in data[:I][n]) -
            sum(vars[:r_D][n,i, ω, t] for i in data[:I][n]) +
            sum(vars[:L_Shed][n,j, ω, t] for j in data[:J][n]) +
            # n ! and W_realized and Q changed
            sum(data[:W_realized][n][q][ω][t] -vars[:W_s][n,q,t] - vars[:W_spill][n,q,ω,t] for q in data[:Q][n]) -
            sum(vars[:PF_scenario][ω, n, ℓ, t] for ℓ in data[:Λ][n]) +
            sum(vars[:PF][n, ℓ, t] for ℓ in data[:Λ][n]) == 0 
    end)
end;


In [112]:
#W_realized[1,1,1,1]
#print(vars[:W_s][1,1,1])
#println(data[:Q])
#println(keys(vars[:W_s]))
data[:b][1][2]


LoadError: KeyError: key :b not found

In [113]:
function init_obj_function(model, data, vars)
    @objective(model, Min, sum(vars[:P][n,i,t]* data[:C][n][i] + vars[:R_U][n,i,t] * data[:C_RU][n][i] + vars[:R_D][n,i,t] * data[:C_RD][n][i] for n in data[:N], i in data[:I][n], t in data[:T]) +
    sum(π[ω] * (sum(data[:C_U][n][i] * vars[:r_U][n,i, ω ,t] + data[:C_D][n][i] * vars[:r_D][n,i, ω, t] for i in data[:I][n]) +
                sum(data[:C_cur][n][q] * vars[:W_spill][n,q, ω, t] for q in data[:Q][n]) +
                sum(data[:V_LOL][n][j] * vars[:L_Shed][n, j, ω, t] for j in data[:J][n])) for n in data[:N], ω in data[:Ω], t in data[:T])
    )
end;

In [114]:
model=Model(HiGHS.Optimizer)
vars=init_variables(model)
init_constraints(model, data, vars)
init_obj_function(model, data, vars)
print(model)

LoadError: At In[110]:2: `@variable(model, P[n in data[:N], i in (data[:I])[n], t in data[:T]] >= 0)`: unexpected error parsing reference set: data[:N]

In [115]:
optimize!(model)

Model   status      : Empty
HiGHS run time      :          0.00


In [116]:
# Print the values of the variables
println("Variable values:")
println("P:")
for n in data[:N], i in data[:I], t in data[:T]
    println("P[$i]: ", JuMP.value.(vars[:P][n, i, t]))
end

println("\nR_U:")
for i in data[:I], t in data[:T]
    println("R_U[$i]: ", value(vars[:R_U][i, t]))
end

println("\nR_D:")
for i in data[:I], t in data[:T]
    println("R_D[$i]: ", value(vars[:R_D][i, t]))end

println("\nL_Shed:")
for j in data[:J], ω in data[:Ω], t in data[:T]
    println("L_Shed[$j, $ω, $t]: ", value(vars[:L_Shed][j, ω, t]))
end

println("\nW_spill:")
for q in data[:Q], ω in data[:Ω], t in data[:T]
    println("W_spill[$q, $ω, $t]: ", value(vars[:W_spill][q, ω, t]))
end

println("\nW_realized:")
for q in data[:Q], ω in data[:Ω], t in data[:T]
    println("W_realized[$q, $ω, $t]: ", value(data[:W_realized][q, ω, t]))
end

println("\nW_s:")
for q in data[:Q], t in data[:T]
    println("W_s[$q, $t]: ", value(vars[:W_s][q, t]))
end

println("\nr_U:")
for i in data[:I], ω in data[:Ω], t in data[:T]
    println("r_U[$i, $ω, $t]: ", value(vars[:r_U][i, ω, t]))
end

println("\nr_D:")
for i in data[:I], ω in data[:Ω], t in data[:T]
    println("r_D[$i, $ω, $t]: ", value(vars[:r_D][i, ω, t]))
end

println("\nLoad")
for j in data[:J]
    print("L[$j]:", data[:L][j])
end

print("\n\nsum:", sum(data[:L]))

Variable values:
P:


LoadError: KeyError: key :N not found